# Assignment 1: N-Gram Language Modeling

| Name               | Student ID |
|--------------------|------------|
| Abdulaziz Alfaraj  | 0   |
| Abu Monguno        | 0   |
| Adeeb Katib        | 0   |
| Ryan Primadi       | 06034343   |


The **Berkeley Restaurant Project (BeRP)** corpus is a speech-dialogue dataset collected in the early 1990s to support a conversational system for answering restaurant‐related questions in Berkeley, California. The dataset contains nearly 9000 user utterances and more than 1700 vocabularies. In this assignment you will implement bigram and trigram language models on these queries, compute the probability of given sentences, and complete given prefixes with greedy decoding. Follow the hints in each code‐block title to fill in the missing parts of each function, then run each code-block to complete the assignment. Ensure your virtual environment is activated before launching the Jupyter Notebook. For detailed instructions, see `environment_setup.md`.

## Step 1: Download the dataset
Run the following code block. It automatically downloads the dataset (stored in `berp_dataset.txt`) and output some summary statistics.

In [2]:
from docs.data_processing import download_berp

download_berp()


download complete
{
  "num_sentences": 8553,
  "num_tokens": 70517,
  "vocab_size": 1749
}


## Step 2: Implement the N-gram Language Model Class

Natural language data is sparse: many valid word sequences never appear in your training corpus, which would give them zero probability under a naïve count-based model. To address this, we apply smoothing when computing probability distributions:

- **Laplace (add-one) smoothing**  
  Add 1 to each N-gram count, and increase each prefix count by the vocabulary size V.  

- **Additive smoothing**  
  More flexible: add a small constant α (e.g. 0.1) to each count, and increase each prefix count by αV:  

When generating text, you convert these probabilities into a choice. We implement two sampling methods:

- **Temperature sampling** re-scales the distribution by raising each probability to the power of 1/T before renormalizing. T > 1 flattens the distribution (more exploration), while T < 1 sharpens it (more exploitation).

- **Greedy sampling** always selects the highest-probability next word, equivalent to temperature sampling in the limit as T $\to$ 0, which collapses the distribution to its single most likely outcome.



Below is the skeleton of the `NGram_LM` class. **Do not** change any existing lines, only fill in the sections marked: 
```
############## YOUR CODE HERE ##############
#                                           
#                                           
############## YOUR CODE HERE ##############
```
Some hints are provided above the marked sections.

In [ ]:
from docs.data_processing import load_data
from collections import defaultdict
from typing import List, Tuple, Optional
import numpy as np


corpus = load_data()


class NGram_LM:
    """
    An N-gram language model 

    Usage example:
      corpus = load_data()
      model = NGram_LM(corpus, N=3)
      prob_of_to = model.next_word_prob(next_word = 'to', prefix = ['i', 'want'])
      generated_next_word = model.next_word_generation(prefix = ['i', 'want'], greedy = True)
    """

    def __init__(
        self,
        corpus: List[List[str]],
        N: int = 2,
    ):
        self.N = N
        self.corpus = corpus
        self.ngram_counts = defaultdict(int)
        
        
        # Count occurrences of each N-gram in the corpus
        # After counting, self.ngram_counts should map each N-gram tuple to its frequency
        # For example, when N=2:
        #   {('see', 'i'): 3,
        #    ('i', 'want'): 773,
        #    ('want', 'to'): 556, ...}
        ############## YOUR CODE HERE ##############
        
        # code for counting N-grams is below inside the same loop to calculate prefix counts      
                  
        ############## YOUR CODE HERE ##############

        self.prefix_counts = defaultdict(int)
        
        # Count occurrences of each (N-1)-gram prefix
        # After counting, self.prefix_counts should map each prefix tuple to its frequency
        # For example, when N=2:
        #   {('okay',): 152,
        #    ("let's",): 271,
        #    ('see',): 84,
        #    ('i',): 2844, ...}
        ############## YOUR CODE HERE ##############
              
        for sentence in self.corpus:
            for i, word in enumerate(sentence):
                ngram = tuple(sentence[i:i + self.N])
                
                # ngram prefix is the first N-1 words of the ngram
                # e.g. if N=2, ngram_prefix = (sentence[i],)
                ngram_prefix = ngram[:-1]
                
                # Count the prefix
                self.prefix_counts[ngram_prefix] += 1
                
                # Count the ngram
                if len(ngram) == self.N:
                    self.ngram_counts[ngram] += 1
        
        ############## YOUR CODE HERE ##############


        # Vocabulary size
        self.vocab = list(set(word for sentence in self.corpus for word in sentence))
        self.vocab_size = len(self.vocab)

    def next_word_prob(self, next_word: str, prefix: List[str], smoothing: str = 'laplace') -> float:
        """Compute next word probability given prefix."""
        assert len(prefix) == self.N - 1, "Prefix must be of length N-1"
        prefix = tuple(prefix)
        ngram = prefix + (next_word,)


        # Compute smoothed probability P(next_word | prefix)
        # Use self.ngram_counts, self.prefix_counts, and the selected smoothing method
        # the smoothing methods are provided in _laplace_smoothing and _additive_smoothing
        ############## YOUR CODE HERE ##############
        
        next_word_probability = 0.0
        
        if smoothing == 'laplace':
            next_word_probability = self._laplace_smoothing(self.ngram_counts[ngram], self.prefix_counts[prefix])
        elif smoothing == 'additive':
            next_word_probability = self._additive_smoothing(self.ngram_counts[ngram], self.prefix_counts[prefix])
        else:
            raise ValueError("Unsupported smoothing method. Use 'laplace' or 'additive'.")
        
        ############## YOUR CODE HERE ##############

        return next_word_probability
        

    def next_word_generation(self, prefix: list, temperature: float = 1.0, greedy: bool = False, smoothing: str = 'laplace', seed: int = 1234):
        
        assert len(prefix) == self.N - 1, "Prefix must be of length N-1" 
        if seed is not None:
            np.random.seed(seed)
        prefix = tuple(prefix)
        
        
        # sample the next word conditional on the prefix
        # you need to implement the temperature sampling and greedy sampling
        # return the sampled next_word
        ############## YOUR CODE HERE ##############
        
        prob = np.array([self.next_word_prob(word, prefix, smoothing) for word in self.vocab])
        
        if greedy:
            next_word = self.vocab[np.argmax(prob)]
        else:
        # re-scales the distribution by raising each probability to the power of 1/T before renormalizing. 
        # T > 1 flattens the distribution (more exploration), while T < 1 sharpens it (more exploitation).
            # normalise the probabilities
            norm_prob = prob / np.sum(prob)
            
            # rescale to the power of 1/T before renormalizing
            rescale = np.power(norm_prob, 1.0 / temperature)  
            
            # renomrlise the probabilities
            re_norm = rescale / np.sum(rescale)  
            
            # sample the next word from the vocabulary based on the re-normalized probabilities
            next_word = np.random.choice(self.vocab, p=re_norm)
            
        ############## YOUR CODE HERE ##############

        return next_word



    def _laplace_smoothing(self, ngram_count: int, prefix_count: int) -> float:
        """Laplace smoothing (add-one)."""
        return (ngram_count + 1) / (prefix_count + self.vocab_size)

    def _additive_smoothing(self, ngram_count: int, prefix_count: int, alpha: float = 0.1) -> float:
        """Additive smoothing with parameter alpha."""
        return (ngram_count + alpha) / (prefix_count + alpha * self.vocab_size)


# check the result
bigram_model = NGram_LM(corpus, N=2)
trigram_model = NGram_LM(corpus, N=3)
print(trigram_model.next_word_prob(next_word = 'to', prefix = ['i', 'want']))
print(trigram_model.next_word_generation(prefix = ['i', 'want'], greedy = True))


0.22030075187969925
to


## Step 3: Sentence Probability Evaluation

Implement a function that computes the log-probability of a full sentence under an N-gram model. Then, using both bigram (N=2) and trigram (N=3) versions of your `NGram_LM`, calculate the log-probability of each sentence given the prefix `['i', 'want']`:

1. "i want to eat on saturday night"  
2. "i want to go to a restaurant"  
4. "i want its sushis"  
5. "i want to eat on monday berkeley"

In [ ]:
def sentence_log_probability(
    model: NGram_LM,
    prefix: List[str],
    sentence: List[str],
    smoothing: str = 'laplace'
) -> float:
    
    # Compute the total log-probability of `sentence` under `model`, starting from the given `prefix` and using the specified smoothing.
    # Hint: accumulate log(next_word_prob) for each word in `sentence`
    ############## YOUR CODE HERE ##############
    
    #initialize log probability
    log_prob = 0.0
    # keep track of the prefix history
    prefix_history = prefix.copy()
    
    # print(f"Calculating log probability for sentence: {' '.join(sentence)}")
    for word in sentence:
        # get the prefix of the last N-1 words
        current_prefix = prefix_history[-(model.N - 1):] 
        
        # print(f"Current prefix: {current_prefix}, Next word: {word}")
        
        # calculate the probability of the next word given the current prefix
        prob = model.next_word_prob(word, current_prefix, smoothing)
        
        # print(f"Probability of '{word}' given prefix {current_prefix}: {prob:.4f}")

        log_prob += np.log(prob)
        
        prefix_history.append(word)
    ############## YOUR CODE HERE ##############
    return log_prob


prefix = ['i', 'want']
sentences = [
    "i want to eat on saturday night",
    "i want to go to a restaurant",
    "i want its sushis",
    "i want to eat on monday berkeley",
]

for model_name, model in [('Bigram', bigram_model), ('Trigram', trigram_model)]:
    print(f"--- {model_name} Model ---")
    for s in sentences:
        words = s.split()[len(prefix):]
        log_p = sentence_log_probability(model, prefix, words)
        print(f"Sentence: '{s}'\nLog-Probability: {log_p:.4f}\n")

--- Bigram Model ---
Sentence: 'i want to eat on saturday night'
Log-Probability: -13.7088

Sentence: 'i want to go to a restaurant'
Log-Probability: -13.7634

Sentence: 'i want its sushis'
Log-Probability: -15.3997

Sentence: 'i want to eat on monday berkeley'
Log-Probability: -17.7053

--- Trigram Model ---
Sentence: 'i want to eat on saturday night'
Log-Probability: -15.8493

Sentence: 'i want to go to a restaurant'
Log-Probability: -15.1558

Sentence: 'i want its sushis'
Log-Probability: -15.3517

Sentence: 'i want to eat on monday berkeley'
Log-Probability: -19.1640



## Step 4: Next-Word Generation with Greedy and Temperature Sampling

Create a function that, given a prefix, produces a sequence of next words using your `NGram_LM`. Then, for both the bigram (N=2) and trigram (N=3) models, generate five-word continuations of the prefix `['i', 'want']` using (a) greedy sampling and (b) temperature-based sampling.


In [ ]:
import numpy as np

def m_step_sentence_generation(model: NGram_LM, prefix: list, m: int, greedy: bool = False, temperature: float = 1.0, seed: int = 1234):
    
    
    # Generate m next words from `model`, starting from `prefix` and using the specified sampling method
    ############## YOUR CODE HERE ##############
    
    prefix_history = prefix.copy()
    
    for _ in range(m):
        current_prefix = prefix_history[-(model.N - 1):]
        generated_word = model.next_word_generation(prefix=current_prefix,greedy=greedy, temperature=temperature, seed=seed)
        prefix_history.append(generated_word)
    generated_sentence = prefix_history
    
    ############## YOUR CODE HERE ##############
    return generated_sentence[len(list(prefix)):]


prefix = ['i', 'want']
m = 5
seeds = [1, 2, 3]

# Scenarios: (description, model, greedy flag, temperature)
tests = [
    ("Bigram (greedy)",     bigram_model, True,  0.5),
    ("Bigram (temperature=0.5)", bigram_model, False, 0.5),
    ("Trigram (greedy)",     trigram_model, True,  0.5),
    ("Trigram (temperature=0.5)",trigram_model, False, 0.5),
]

for desc, model, greedy, temp in tests:
    print(f"--- {desc} ---")
    for seed in seeds:
        generated = m_step_sentence_generation(
            model=model,
            prefix=prefix.copy(),
            m=m,
            greedy=greedy,
            temperature=temp,
            seed=seed
        )
        full_sentence = prefix + generated
        print(f"Seed {seed}: {' '.join(full_sentence)}")
    print()



--- Bigram (greedy) ---
Seed 1: i want to eat on saturday night
Seed 2: i want to eat on saturday night
Seed 3: i want to eat on saturday night

--- Bigram (temperature=0.5) ---
Seed 1: i want to eat on wednesday particularly
Seed 2: i want to eat on wednesday come
Seed 3: i want to eat lunch nineteen (i)

--- Trigram (greedy) ---
Seed 1: i want to eat on a sunday
Seed 2: i want to eat on a sunday
Seed 3: i want to eat on a sunday

--- Trigram (temperature=0.5) ---
Seed 1: i want to eat on style croissant
Seed 2: i want to eat on sundays screaming
Seed 3: i want to eat lunch midprice either

